In [1]:
from os import chdir
chdir('/Users/lananhnguyen/Desktop/thesis/thesis_code')
from packages.load import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
import xgboost as xgb
from lightgbm import LGBMRegressor
import optuna

In [2]:
hicp_all_path = 'data/preprocessed/hicp_yoy.csv'
hicp_class_path = 'data/HICP_COICOP10s.xlsx'
hicp_cat_path = 'data/preprocessed/food_yoy_infl.csv'
category = 'Food'

HICP_monthly, HICP_class, HICP_cat = import_data_all(hicp_all_path=hicp_all_path,
                                                     hicp_class_path=hicp_class_path,
                                                     hicp_cat_path=hicp_cat_path)

cat_df = split_into_category(category=category,
                             HICP_class=HICP_class,
                             HICP_monthly=HICP_monthly)

X_cat_train, y_cat_train = split_train_set(cat_df=cat_df,
                                           HICP_cat=HICP_cat,
                                           h = 1)

Number of items in Food group:  180
Training predictor period: 1997-01-31 00:00:00 to 2015-11-30 00:00:00
Training dependent variable period: 1997-02-28 00:00:00 to 2015-12-31 00:00:00


/Users/lananhnguyen/Desktop/thesis/thesis_code/main/packages/load.py:237: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_df.fillna(0, inplace=True)


In [5]:
import datetime
from optuna_dashboard import run_server

storage = optuna.storages.InMemoryStorage()

# 1. Define an objective function to be maximized.
def objective(trial):

    params = {
        'objective': 'reg:squarederror',
        'booster': 'gbtree',
        'eval_metric': 'rmse',
        'tree_method': 'auto',
        'n_jobs': -1,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'gamma': trial.suggest_float('gamma', 0.0, 1.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 10.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'lambda': trial.suggest_float('lambda', 0.01, 1),
        'alpha': trial.suggest_float('alpha', 0.01, 0.5)
    }
    tscv = TimeSeriesSplit(n_splits=5)
    model = xgb.XGBRegressor(**params)

    # Use cross_val_score to get the mean RMSE over multiple splits
    scores = -cross_val_score(model, X_cat_train, y_cat_train, cv=tscv, scoring='neg_root_mean_squared_error', n_jobs=-1)
    
    return scores.mean()

# 3. Create a study object and optimize the objective function.
study = optuna.create_study(
    direction='minimize',
    storage=storage
)
study.optimize(objective, n_trials=10)
run_server(storage, host="localhost", port=8080)
best_params = study.best_params
best_trial = study.best_trial

[I 2024-03-12 21:27:47,217] A new study created in memory with name: no-name-4a07018c-b2b7-475f-85e8-43a2a4a11d53
[I 2024-03-12 21:27:49,370] Trial 0 finished with value: 1.410758393396035 and parameters: {'learning_rate': 0.15276219936260296, 'max_depth': 5, 'gamma': 0.43122589068865147, 'min_child_weight': 7.67595760524476, 'n_estimators': 152, 'lambda': 0.24656000006524806, 'alpha': 0.23302784159334544}. Best is trial 0 with value: 1.410758393396035.
[I 2024-03-12 21:27:50,476] Trial 1 finished with value: 1.4041896903901514 and parameters: {'learning_rate': 0.2719558594825998, 'max_depth': 10, 'gamma': 0.07591852657785425, 'min_child_weight': 5.931700577330134, 'n_estimators': 98, 'lambda': 0.6755867646555919, 'alpha': 0.4638901464999106}. Best is trial 1 with value: 1.4041896903901514.
[I 2024-03-12 21:27:51,319] Trial 2 finished with value: 1.412294609851031 and parameters: {'learning_rate': 0.2828905899627169, 'max_depth': 11, 'gamma': 0.735049018155555, 'min_child_weight': 4.49

In [6]:
best_params

{'learning_rate': 0.2719558594825998,
 'max_depth': 10,
 'gamma': 0.07591852657785425,
 'min_child_weight': 5.931700577330134,
 'n_estimators': 98,
 'lambda': 0.6755867646555919,
 'alpha': 0.4638901464999106}